In [76]:
import openai
import os, re, calendar, requests
from datetime import datetime, timedelta
from dotenv import load_dotenv
from pathlib import Path
load_dotenv("key.env")
KTO_API_KEY = os.getenv("TOUR_API_KEY_D")
openai.api_key = os.getenv("OPENAI_API_KEY")


In [77]:
# 날짜 파싱함수
def format_range(start, end):
    return start.strftime("%Y%m%d"), end.strftime("%Y%m%d")

def parse_season(text, today):
    season_map = {
        "봄": (3, 5), "여름": (6, 8),
        "가을": (9, 11), "겨울": (12, 2)
    }
    for season, (start_m, end_m) in season_map.items():
        if season in text:
            if season == "겨울":
                start = datetime(today.year, 12, 1)
                end = datetime(today.year + 1, 2, calendar.monthrange(today.year + 1, 2)[1])
            else:
                start = datetime(today.year, start_m, 1)
                end = datetime(today.year, end_m, calendar.monthrange(today.year, end_m)[1])
            return format_range(start, end)
    return None

def parse_month(text, today):
    match = re.search(r"(\d{1,2})월(?:\s*(전체|한 달)?)?", text)
    if match:
        m = int(match.group(1))
        y = today.year + (1 if m < today.month else 0)
        start = datetime(y, m, 1)
        end = datetime(y, m, calendar.monthrange(y, m)[1])
        return format_range(start, end)
    return None

def parse_range(text, today):
    match = re.search(r"(\d{1,2})월\s*(\d{1,2})일\s*부터\s*(\d{1,2})월\s*(\d{1,2})일", text)
    if match:
        sm, sd, em, ed = map(int, match.groups())
        sy = today.year + (1 if sm < today.month else 0)
        ey = sy if sm <= em else sy + 1
        start = datetime(sy, sm, sd)
        end = datetime(ey, em, ed)
        return format_range(start, end)
    return None

def parse_relative(text, today):
    base = today
    if "내일" in text: base += timedelta(days=1)
    elif "모레" in text: base += timedelta(days=2)

    if "오늘부터 일주일" in text:
        return format_range(base, base + timedelta(days=6))
    elif "이번 주말" in text:
        start = base + timedelta(days=(5 - base.weekday()))
        return format_range(start, start + timedelta(days=1))
    elif "다음 주말" in text:
        start = base + timedelta(days=(5 - base.weekday() + 7))
        return format_range(start, start + timedelta(days=1))
    elif "이번 달" in text:
        start = base.replace(day=1)
        end = base.replace(day=calendar.monthrange(base.year, base.month)[1])
        return format_range(start, end)
    elif "다음 달" in text:
        nm = base.month % 12 + 1
        ny = base.year + (base.month == 12)
        start = datetime(ny, nm, 1)
        end = datetime(ny, nm, calendar.monthrange(ny, nm)[1])
        return format_range(start, end)
    elif match := re.search(r"(\d+)\s*일", text):
        d = int(match.group(1))
        return format_range(base, base + timedelta(days=d - 1))
    return None

def parse_exact_date(text, today):
    match = re.search(r"(?:(\d{1,2})월)?\s*(\d{1,2})일", text)
    if match and "부터" not in text:
        m = int(match.group(1)) if match.group(1) else today.month
        d = int(match.group(2))
        y = today.year + (1 if m < today.month else 0)
        dt = datetime(y, m, d)
        return format_range(dt, dt)
    return None

def parse_date_range_from_text(text: str):
    today = datetime.today()
    for parser in [parse_range, parse_month, parse_season, parse_relative, parse_exact_date]:
        if result := parser(text, today):
            return result
    return None, None


In [82]:
def safe_request(url, params):
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()

        # 응답이 JSON 형식인지 확인 (방어 추가)
        if 'application/json' not in response.headers.get("Content-Type", ""):
            print("[응답 형식 오류] JSON이 아닌 응답입니다.")
            return {}

        return response.json()
    except requests.RequestException as e:
        print(f"[API 요청 오류] {e}")
        return {}
    except Exception as e:
        print(f"[응답 파싱 오류] {e}")
        return {}
# 지역코드
def get_area_codes(region_name, subregion_name=None):
    url = "https://apis.data.go.kr/B551011/KorService2/areaCode2"
    params = {
        "serviceKey": KTO_API_KEY,
        "MobileOS": "ETC",
        "MobileApp": "TourChat",
        "_type": "json",
        "numOfRows": 100
    }

    data = safe_request(url, params)
    items = data.get("response", {}).get("body", {}).get("items", {}).get("item", [])
    region_code = next((r["code"] for r in items if region_name in r["name"]), None)

    subregion_code = None
    if region_code and subregion_name:
        params["areaCode"] = region_code
        subitems = safe_request(url, params).get("response", {}).get("body", {}).get("items", {}).get("item", [])
        subregion_code = next((s["code"] for s in subitems if subregion_name in s["name"]), None)

    return region_code, subregion_code
# 행사
def get_events(start_date, end_date, area_code=None, sigungu_code=None):
    url = "https://apis.data.go.kr/B551011/KorService2/searchFestival2"
    params = {
        "serviceKey": KTO_API_KEY,
        "MobileOS": "ETC",
        "MobileApp": "TourChat",
        "eventStartDate": start_date,
        "eventEndDate": end_date,
        "areaCode": area_code,
        "sigunguCode": sigungu_code,
        "numOfRows": 50,
        "_type": "json"
    }

    data = safe_request(url, params)
    if not isinstance(data, dict):
        print("[경고] API 응답이 dict 형식이 아닙니다:", type(data))
        return []
    return data.get("response", {}).get("body", {}).get("items", {}).get("item", [])

In [115]:
def web_search(query):
    messages = [
        {"role": "system", "content": "웹 검색 결과를 3가지로 요약해줘."},
        {"role": "user", "content": query}
    ]
    try:
        response = openai.chat.completions.create(
            model="gpt-4.1",
            messages=messages,
            max_tokens=150,
            temperature=0.7,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"웹서치 중 오류 발생: {e}"


In [94]:
# 행사 필터링 함수
def filter_events_by_date(events, trip_start, trip_end):
    filtered = []
    fmt = "%Y%m%d"
    ts = datetime.strptime(trip_start, fmt)
    te = datetime.strptime(trip_end, fmt)

    for e in events:
        try:
            es = datetime.strptime(e.get("eventstartdate", ""), fmt)
            ee = datetime.strptime(e.get("eventenddate", ""), fmt)
            if es <= te and ee >= ts:
                filtered.append(e)
        except Exception:
            continue
    return filtered


In [116]:
def chatbot_query(text: str):
    # 날짜 범위 파싱
    start, end = parse_date_range_from_text(text)
    if not start or not end:
        return "여행 기간을 인식하지 못했어요. 날짜를 다시 입력해 주세요."

    # 지역 정보 추출
    match = re.search(r"(서울|부산|대구|울산|광주|대전|인천|세종|제주|제주도|경기|강원|충북|충남|전북|전남|경북|경남)(\s*(\S+))?", text)
    region = match.group(1) if match else None
    subregion = match.group(3).strip() if match and match.group(3) else None

    if not region:
        return "지역 정보를 찾을 수 없어요. 예: '부산', '서울 강남'"

    # 지역 코드 조회
    area_code, sigungu_code = get_area_codes(region, subregion)
    if not area_code:
        return f"{region} 지역 정보가 없습니다."

    # 행사 데이터 가져오기
    events = get_events(
        start_date=start,
        end_date=end,
        area_code=str(area_code),
        sigungu_code=str(sigungu_code) if sigungu_code else None
    )

    # 날짜 기준으로 필터링
    events = filter_events_by_date(events, start, end)

    if not events:
        return f"{region} 지역에서 {start} ~ {end} 기간 동안 열리는 행사가 없습니다."

    # 행사 결과 메시지 작성
    reply = f"{region} 행사 정보 ({start} ~ {end})\n\n"
    for event in events[:3]:
        title = event.get("title", "제목 없음")
        addr = event.get("addr1", "주소 미확인")
        tel = event.get("tel", "전화번호 없음")
        evt_start = event.get("eventstartdate", "")
        evt_end = event.get("eventenddate", "")
        img_url = event.get("firstimage", "")

        sf = f"{evt_start[:4]}.{evt_start[4:6]}.{evt_start[6:]}" if evt_start else ""
        ef = f"{evt_end[:4]}.{evt_end[4:6]}.{evt_end[6:]}" if evt_end else ""

        reply += f"{title}\n{addr}\n전화: {tel}\n{sf} ~ {ef}\n"
        reply += f"포스터: {img_url if img_url else '없음'}\n\n"

    # 행사 관련 웹서치 결과 추가
    search_query = f"{region} {start}부터 {end}까지 행사"
    search_result = web_search(search_query)
    reply += "\n[웹 검색 요약 결과]\n" + search_result

    return reply


In [117]:
if __name__ == "__main__":
    query = "부산 10월 놀러갈건데 어디가지?"
    result = chatbot_query(query)
    print(result)

부산 행사 정보 (20251001 ~ 20251031)

광안리 M(Marvelous) 드론 라이트쇼
부산광역시 수영구 광안해변로 219
전화: 051-610-4882
2022.04.02 ~ 2025.12.31
포스터: http://tong.visitkorea.or.kr/cms/resource/69/2786369_image2_1.jpg

동래읍성역사축제
부산광역시 동래구 문화로 80
전화: 051-550-4092
2025.10.24 ~ 2025.10.26
포스터: http://tong.visitkorea.or.kr/cms/resource/43/3366443_image2_1.jpg

라라라 페스티벌
부산광역시 금정구 장전온천천로 48 (장전동)
전화: 051-519-4422
2025.10.24 ~ 2025.10.26
포스터: http://tong.visitkorea.or.kr/cms/resource/03/3405703_image2_1.jpg


[웹 검색 요약 결과]
웹 검색 결과가 제공되지 않았으므로, 부산에서 2025년 10월 1일부터 10월 31일까지 개최될 가능성이 높은 주요 행사를 일반적인 정보와 예시로 3가지로 요약해드립니다.

1. 부산국제영화제 (Busan International Film Festival, BIFF)
   - 세계적으로 유명한 영화제로, 매년 10월 부산 해운대와 영화의전당 등에서 개최됩니다. 다양한 국내외 영화 상영, 감독/배우와의 만남, 이벤트 등이 진행됩니다.

2. 부산불꽃축제
   - 부산 광안리 해수욕장 일대에서 개최되는 대규모 불꽃축제입니다
